# k-Means Clustering

In the previous few sections, we have explored one category of unsupervised machine learning models: dimensionality reduction. Here we will move on to another class of unsupervised machine learning models: clustering algorithms. Clustering algorithms seek to learn, from the properties of the data, an optimal division or discrete labeling of groups of points.

Many clustering algorithms are available in Scikit-Learn and elsewhere, but perhaps the simplest to understand is an algorithm known as k-means clustering, which is implemented in sklearn.cluster.KMeans. We begin with the standard imports:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()    # for plot styling
import numpy as np

# Introducing k-Means

The k-means algorithm searches for a predetermined number of clusters within an unlabeled multidimensional dataset. It accomplishes this using a simple conception of what the optimal clustering looks like:

- The “cluster center” is the arithmetic mean of all the points belonging to the cluster.
- Each point is closer to its own cluster center than to other cluster centers.

Those two assumptions are the basis of the k-means model. We will soon dive into exactly how the algorithm reaches this solution, but for now let’s take a look at a simple dataset and see the k-means result.

First, let’s generate a two-dimensional dataset containing four distinct blobs. To emphasize that this is an unsupervised algorithm, we will leave the labels out of the visualization (Figure 5-110):

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

By eye, it is relatively easy to pick out the four clusters. The k-means algorithm does this automatically, and in Scikit-Learn uses the typical estimator API:

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

> **Figure 5-110: _Data for demonstration of clustering_**<br><img width="300" src="data:image/png;base64, ..." />

Let’s visualize the results by plotting the data colored by these labels. We will also plot the cluster centers as determined by the k-means estimator (Figure 5-111):

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

> **Figure 5-111: _k-means cluster centers with clusters indicated by color_**<br><img width="300" src="data:image/png;base64, ..." />

The good news is that the k-means algorithm (at least in this simple case) assigns the points to clusters very similarly to how we might assign them by eye. But you might wonder how this algorithm finds these clusters so quickly! After all, the number of possible combinations of cluster assignments is exponential in the number of data points - an exhaustive search would be very, very costly. Fortunately for us, such an exhaustive search is not necessary; instead, the typical approach to k-means involves an intuitive iterative approach known as expectation-maximization.

# k-Means Algorithm: Expectation-Maximization

Expectation-maximization (E-M) is a powerful algorithm that comes up in a variety of contexts within data science. k-means is a particularly simple and easy-to-understand application of the algorithm, and we will walk through it briefly here. In short, the expectation-maximization approach consists of the following procedure:

1. Guess some cluster centers
2. Repeat until converged
  1. E-Step: assign points to the nearest cluster center
  2. M-Step: set the cluster centers to the mean

Here the “E-step” or “Expectation step” is so named because it involves updating our expectation of which cluster each point belongs to. The “M-step” or “Maximization step” is so named because it involves maximizing some fitness function that defines the location of the cluster centers - in this case, that maximization is accomplished by taking a simple mean of the data in each cluster.

The literature about this algorithm is vast, but can be summarized as follows: under typical circumstances, each repetition of the E-step and M-step will always result in a better estimate of the cluster characteristics.

We can visualize the algorithm as shown in Figure 5-112.

For the particular initialization shown here, the clusters converge in just three iterations. For an interactive version of this figure, refer to the code in the online appendix.

> **Figure 5-112: _Visualization of the E-M algorithm for k-means_**<br><img width="300" src="data:image/png;base64, ..." />

The k-means algorithm is simple enough that we can write it in a few lines of code. The following is a very basic implementation (Figure 5-113):

In [ ]:
from sklearn.metrics import pairwise_distances_argmin
def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
         # 2b. Find new centers from means of points
         new_centers = np.array([X[labels == i].mean(0)
                                 for i in range(n_clusters)])
         # 2c. Check for convergence
         if np.all(centers == new_centers):
             break
         centers = new_centers
    return centers, labels
centers, labels = find_clusters(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

> **Figure 5-113: _Data labeled with k-means_**<br><img width="300" src="data:image/png;base64, ..." />

Most well-tested implementations will do a bit more than this under the hood, but the preceding function gives the gist of the expectation-maximization approach.

# Caveats of expectation-maximization

There are a few issues to be aware of when using the expectation-maximization algorithm.

# The globally optimal result may not be achieved

First, although the E-M procedure is guaranteed to improve the result in each step, there is no assurance that it will lead to the global best solution. For example, if we use a different random seed in our simple procedure, the particular starting guesses lead to poor results (Figure 5-114):

In [ ]:
centers, labels = find_clusters(X, 4, rseed=0)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

> **Figure 5-114: _An example of poor convergence in k-means_**<br><img width="300" src="data:image/png;base64, ..." />

Here the E-M approach has converged, but has not converged to a globally optimal configuration. For this reason, it is common for the algorithm to be run for multiple starting guesses, as indeed Scikit-Learn does by default (set by the n_init parameter, which defaults to 10).

# The number of clusters must be selected beforehand

Another common challenge with k-means is that you must tell it how many clusters you expect: it cannot learn the number of clusters from the data. For example, if we ask the algorithm to identify six clusters, it will happily proceed and find the best six clusters (Figure 5-115):

In [ ]:
labels = KMeans(6, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

> **Figure 5-115: _An example where the number of clusters is chosen poorly_**<br><img width="300" src="data:image/png;base64, ..." />

Whether the result is meaningful is a question that is difficult to answer definitively; one approach that is rather intuitive, but that we won’t discuss further here, is called silhouette analysis.

Alternatively, you might use a more complicated clustering algorithm which has a better quantitative measure of the fitness per number of clusters (e.g., Gaussian mixture models; see “In Depth: Gaussian Mixture Models” on page 476) or which can choose a suitable number of clusters (e.g., DBSCAN, mean-shift, or affinity propagation, all available in the sklearn.cluster submodule).

# k-means is limited to linear cluster boundaries

The fundamental model assumptions of k-means (points will be closer to their own cluster center than to others) means that the algorithm will often be ineffective if the clusters have complicated geometries.

In particular, the boundaries between k-means clusters will always be linear, which means that it will fail for more complicated boundaries. Consider the following data, along with the cluster labels found by the typical k-means approach (Figure 5-116):

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)

In [ ]:
labels = KMeans(2, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

> **Figure 5-116: _Failure of k-means with nonlinear boundaries_**<br><img width="300" src="data:image/png;base64, ..." />

This situation is reminiscent of the discussion in “In-Depth: Support Vector Machines” on page 405, where we used a kernel transformation to project the data into a higher dimension where a linear separation is possible. We might imagine using the same trick to allow k-means to discover nonlinear boundaries.

One version of this kernelized k-means is implemented in Scikit-Learn within the SpectralClustering estimator. It uses the graph of nearest neighbors to compute a higher-dimensional representation of the data, and then assigns labels using a k-means algorithm (Figure 5-117):

In [ ]:
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2,
                           affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

We see that with this kernel transform approach, the kernelized k-means is able to find the more complicated nonlinear boundaries between clusters.

> **Figure 5-117: _Nonlinear boundaries learned by SpectralClustering_**<br><img width="300" src="data:image/png;base64, ..." />

# k-means can be slow for large numbers of samples

Because each iteration of k-means must access every point in the dataset, the algorithm can be relatively slow as the number of samples grows. You might wonder if this requirement to use all data at each iteration can be relaxed; for example, you might just use a subset of the data to update the cluster centers at each step. This is the idea behind batch-based k-means algorithms, one form of which is implemented in sklearn.cluster.MiniBatchKMeans. The interface for this is the same as for standard KMeans; we will see an example of its use as we continue our discussion.

# Examples

Being careful about these limitations of the algorithm, we can use k-means to our advantage in a wide variety of situations. We’ll now take a look at a couple examples.

# Example 1: k-Means on digits

To start, let’s take a look at applying k-means on the same simple digits data that we saw in “In-Depth: Decision Trees and Random Forests” on page 421 and “In Depth: Principal Component Analysis” on page 433. Here we will attempt to use k-means to try to identify similar digits without using the original label information; this might be similar to a first step in extracting meaning from a new dataset about which you don’t have any a priori label information.

We will start by loading the digits and then finding the KMeans clusters. Recall that the digits consist of 1,797 samples with 64 features, where each of the 64 features is the brightness of one pixel in an 8×8 image:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Output:<br>`
(1797, 64)
        `

The clustering can be performed as we did before:

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits.data)
kmeans.cluster_centers_.shape

Output:<br>`
(10, 64)
        `

The result is 10 clusters in 64 dimensions. Notice that the cluster centers themselves are 64-dimensional points, and can themselves be interpreted as the “typical” digit within the cluster. Let’s see what these cluster centers look like (Figure 5-118):

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = kmeans.cluster_centers_.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

> **Figure 5-118: _Cluster centers learned by k-means_**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAcsAAAC1CAIAAAC/P0MVAAAACXBIWXMAAAsSAAALEgHS3X78AAALuElEQVR42u3da0yW9R/H8R+CnI9Dpw0FFZ1GTWurUDdKazooXTxQVBJr9IBhq60wgnog9ox12FxpG/mEufnACRsFHnPzfJigq3nYojg0UmBxBqE4/Z/77/v57395e4H1fj397Gt3v/u6P167dv38BU1OTjoAwCMwgyUAABoWAGhYAAANCwA0LAA8lkIC+8eNjo6KtLy83IqqqqrEYHV1tUhTUlLMv0Bm+PdXyP37962ouLhYDH777bdWFBkZKQZLSkpE+t5771lRRESEP2vS3d1tRQUFBWLw4sWLVjQ4OCgGk5KSRPrZZ59Z0YYNG/xZk/HxcSvau3evGNy3b58VRUdHi8F3331XpFu3bvX2xwbWxMSEt3L48MMPraizs1MMpqeni7SwsNCK1q9fLwbj4+O5hwUAnhIAAA0LAKBhAYCGBQAaFgBAwwIADQsA/xQB3nHQ0NAg0i+++MKK8vLyxGBYWJhI+/v7rUi/Mh0SEsj//fPnz1vR6dOnxeCOHTusqLGxUQwePnxYpOJNcrFHI7AuXLhgRUeOHBGDy5Yts6Ls7GwxmJycLNJFixZN+U/u9u3bVvTJJ5+IwU2bNlmR2MXwP5d648aN3n4+gdXT02NFBw4cEINz5861orS0NDGof1xiA5GuI+5hAYCnBABAwwIAaFgAoGEBgIYFANCwAEDDAsA/iJdX7oeGhqyorKxMDIrXyDMzM8XgmTNnRCpeOc7IyPBtKcX/nTjFQCsqKhLp4sWLRZqQkDDll9fdu3etKDQ0VAy+//77VrRu3Tox+MC/MP+A2NjYKV+T1tZWK5o1a5YYFDsOxC4G51xdXZ1Ixav1fpqcnLSi3NxcMSiOKvjuu+/EYFdXl7cfV3h4OPewAMBTAgCgYQEANCwA0LAAQMMCAGhYAKBhAYCGBQA8DC97uk6cOOEhcs5VVFRY0c2bN8WgPoVFHIPx8ssv+7aU4mgWvdFoz549VvTLL7+IwZycHJFGRERM+eUl9lAFBweLwa+++sqKWlpaxOBbb70l0ri4uClfE7FZ6OmnnxaDu3fvtqL29nYxqLeKiS2afhLXg95DdezYMSs6ePCg58/T0dFhRUFBQdzDAgBPCQCAhgUA0LAAQMMCAA0LAKBhAYCGBQAaFgDwMLzs6Tp+/Li3/9iFCxes6Pz582JQHPfknHvjjTemw1KKg4ZCQtQ6r1mzxoouXbokBisrK0W6YcMGK1q6dKk/a7Jq1Soreuedd8Tgr7/+akX6TKorV66IdN68eVYUGRnpz5qkpqZakdi15Zyrr6+3osHBQTFYU1Mj0t9++82K9B6zwBoZGbGis2fPisHvv//eitra2sSgPvAtUNv/uIcFAJ4SAAANCwCgYQGAhgUAGhYAQMMCAA0LAI+zv38TXrw875xLS0uzohdeeEEMNjQ0WFFzc7MYXL9+vUhfeeWV6bCUf/31lxUNDAx4+/xhYWFicNeuXSI9deqUFfm240C84a8/vNiccvjwYTGoN6eMj49P+XUiziDRb7lnZWWZP2O5peXo0aMi/fnnn63o1Vdf9W1ZxKX+4osvisEbN25Ykdjc4ZwrLS0V6cqVK7mHBQCeEgAADQsAoGEBgIYFABoWAEDDAgANCwA0LEsAAI+Il1Nk8vLyrCgjI0MMVlVVWVFra6sY/OCDD0SalJQ0HZZSHINx4MABMRgdHW1FTU1NYrCjo0N9tfY+H721KTg42Ic10UeD1NbWWpHYg+Sce/3110UaGho65dfJ8PCwFVVUVIjBp556yori4+PFYGdnp0jFZiq9t1NsTvMgISHBipYsWSIG+/r6rKigoEAMrl692tuycA8LADwlAAAaFgBAwwIADQsANCwAgIYFABoWAP41/v6ldP0ucWJiohXFxMSIQXH+R0pKihh88sknp/9SileU9UvyBw8etKLY2FgxmJubK1Jx6MjY2JgYDOCOg9HRUSvSuzDEfoSioiIxmJmZ6e078k1UVJS3X0FxcbEV9fT0iMHs7GyRrl27djr8fMTuhqtXr4pBsYtkzZo1/lzn3MMCAE8JAICGBQDQsABAwwIADQsAoGEBgIYFABoWAPB/CdIHRQAAuIcFABoWAGhYAAANCwA0LADQsAAAGhYAaFgAoGEBADQsANCwAEDDAgBoWACgYQGAhmUJAOARCQnsH1dbWyvSN99804pSU1PF4P79+0X63HPPPdbfQWtrqxVt3rxZDLa1tYm0oqLCijIzM9U1ERKwq0L868M//PCDGPzoo4+saGRkRAx+/PHHIhXrGRYW5s/X3dvba0Xbt28Xg3V1dVZUXFwsBktKSkSakJAwzX8gXV1d3r7T+fPni8Hy8nKRzp07l3tYAOApAQDQsAAAGhYAaFgAoGEBADQsANCwAPCv5+Xd8r6+PisqLCwUg+Pj41Y0NjYmBgsKCkQq3l33821q8XZ9c3OzGCwrK7Oia9euicF58+aJtKenx8NHDayBgQErOnbsmBgUmyn0d1pTUyPStWvXWlFSUpI/ayJ25Zw7d04MLliwwIpOnjwpBrOzs0W6atWq6dBE4poUe2ecc5cvX7ainJwcMRgZGck9LADwlAAAQMMCAA0LADQsAICGBQAaFgBoWACA5GXHQX19vRXpf3W/srLSivRrz6+99ppIf/rpJyt66aWXfFtK8f/+6aefisHjx49bUXJyshgU7/M75+bMmWNFwcHB/qzJn3/+aUUTExNiUGym0GcchIeHi9Sfl8y16OhoK3r77bfFoDgHZN++fWJwdHR0+jfR4OCgFR06dEgMbtmyxYq2bdsmBmNiYriHBQCeEgAAaFgAoGEBgIYFANCwAEDDAgANCwCgYQFgSnjZ0yU2L+lzTVavXm1F4oQM51x6erpIxVErfu7pamxstKKzZ8+Kwfj4eCvq6OgQg2LHlHPuiSeeMP9eneHT36wzZ860osTERG+LKc4ics7t2bNHpLGxsVP+kxPXZEpKihisq6vzdiX4eZaSZ/fu3bOiP/74QwyKvYvXr18XgwsXLhSp+C6CgoK4hwUAnhIAAA0LAKBhAYCGBQAaFgBAwwIADQsANCwA4GF42dN1//59K5o1a5YYnD17tvk5QtQnEVuDnDzhx09iE0h+fr4YFBtyqqqqxGBvb69IxXlQvhEHgulDxsR3qrf/PfPMM94+j2/EDivxy3LONTQ0eFsT/aucJjo7O62op6dHDFZXV1vRyZMnxaDe01VWVmZFy5cv5x4WAHhKAAA0LACAhgUAGhYAaFgAAA0LADQsAPw7eNlxkJaWZkUtLS1isL+/34qGhobE4I8//ijS559/fjospdhPsW3bNjHY3NxsRXrHQUxMjEijoqKmfE1GRkas6ObNm2JwxYoVVjQxMSEG9RW4aNEiK/JtM8LY2JgViT0Fzrn29nYr0qcl6Z0p4jrx89Cd+fPnW5E+tWXp0qVWlJOTIwZrampE+vXXX1vRN998IwYfuJC4hwUAnhIAAA0LAKBhAYCGBQAaFgBAwwIADQsANCwA4L952dP17LPPmoU9Q1X2zp07rUicpOKca2pqEmlGRsZ0WEqxwyoiIkIMikMy+vr6xGBcXJxIxX4q34SGhlpRfHy8GOzu7rai4eFhMfj777+LVFxmkZGR/qyJ+Lo///xzMXjr1i0r0gcpnTt3TqRiP9iuXbvEYGJiYgCXRezpysrKEoN37tzxVkddXV0iFbtMdVk9cCFxDwsAPCUAABoWAEDDAgANCwA0LACAhgUAGhYAHmdedhyIsyUqKyvF4NatW83PEaI+yZdffilScYyEn8RZFzNnzhSD4viZOXPmiEG9kUGftuIPcans2LFDDJaWllqR3nEQHR0t0vHx8SlfE3Ex6IN/BgYGrKitrc3bBeac6+josCJxvpEL9I4DsTtg//79YjA/P9+K8vLyxOCCBQtEunfvXisKDw/nHhYAeEoAADQsAICGBQAaFgBoWAAADQsANCwA0LAAgIcRNDk5ySoAAPewAEDDAgBoWACgYQGAhgUA0LAA8Kj9Bz0d4Kl6vsQPAAAAAElFTkSuQmCC" />

We see that even without the labels, KMeans is able to find clusters whose centers are recognizable digits, with perhaps the exception of 1 and 8.

Because k-means knows nothing about the identity of the cluster, the 0-9 labels may be permuted. We can fix this by matching each learned cluster label with the true labels found in them:

In [ ]:
from scipy.stats import mode
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

Now we can check how accurate our unsupervised clustering was in finding similar digits within the data:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(digits.target, labels)

Output:<br>`
0.79354479688369506
        `

With just a simple k-means algorithm, we discovered the correct grouping for 80% of the input digits! Let’s check the confusion matrix for this (Figure 5-119):

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(digits.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

> **Figure 5-119: _Confusion matrix for the k-means classifier_**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAWAAAAFgCAIAAAAKAzQbAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2deTxU+//HP2cWY8s29l0pWlW3tCtupVVo0UabrqUolRCSiuhKIYV2UiK03UrFVdIt1G29lxYkwlizL2Pm98d01a90v2U+x52p9/PxfTy+Z87Vy3venzMvn3PmnM+L4HK5CAAAoCso0AIAAMAgAAAAgwAAAAwCAAAwCAAAwCAAABBIaD3/K4doTSRJOedJEol1k/p9MEHAsQgN/28RkWLCDAIAADjFAAAADAIAADAIAADAIAAAAIMAAAAMAgAAMAgAAMAgAAAAgwAAAPgATaCq2RHk/jw3P+ZwfNABXw0tVYQQIgg1deWcuw9Ddh8KCPXm3X5LpVJ19XRc7Lx/v3a7e7/o1u3MkAOR7e3t/XR1t3t7iIuLY38vXtv9+vbps2zJQryypFZOnjg0XEiHUlBmENp9NA+dCp4y4/1jGpscfaxmrraaudrX7df6ugY/r70FL19bzbDl7fwjI/u3cze67Q41tbXeO/z37d51IeG0mqpKcNgBvO8lv/C1raPztdTfsXeJ1MrJE4eGC+9Q4jQIDofT7X+70MYiOf5yym/p/296Q6PuDPYI3BZawarq3Dl85JDJ0yfu9Azu9u+6czdr8IABGupqCCGreRaXr17De1TFJSSam800nWyC/XgltXLyxKHhwjuUGE4x3rx5s2vXrqdPn9JoNA6H069fPw8PDx0dnW8SCfAJQQiNHjfi452WC2exyirTb9z5eOeGLfahuw81NzV3u+Cy8nJlJUXetpKiYmNTU1NTE8YJ3hbXDQihu1k52I9XUisnTxwaLrxDicEgPD09N27caGBgwHv58OFDDw+PuLg4/pWXrpy3ze3Xj/cY/DRQWlb6yoVUfmS5nE+fI6ZQqEJxxYjUyskTh4YL71BiOMVoa2vrdAeE0NChQ7G8Sb0BuhQq5UH24493ms40vpiUwqeysrISq7KSt13OYkn16iUqyhCK45XUyskTh4YL71BiMAg9PT0PD4/Lly9nZGRcvXrVw8NDT0+Pf9kRowyy7vz5yc6fRg+9l3mfT+Wxow2fPP3rTXExQigh6bzxxAlISCC1cvLEoeHCO5QYTjG2bdt248aN+/fvNzQ0SEpKGhsbT5kypZtTI/RhaqSpo/62uOyTH9DUUnv7pozPguVkZXds9XTZ7MlmszXU1fx8vck4tggS1iwitXLyxKHhwjuURM8na8GSc10e3QiAhv+nwJJzAAD0+DUIAADAIAAAAIMAAAAAgwAAAAwCAAAwCAAAwCAAAACDAAAADAIAADAIAACElP/gWYy2uiqSlAvP/U5e2VpmRiQOA5nrI7CbG0lSpolJkHigCOezGB2tLeRV3VpRQZ64zIChMIMAAABOMQAAAIMAAAAMAgAAMAgAAMAgAAAAgwAAAAwCAAAwCAAAfkBoglwc9vjjIzcuZ/79rJeYGEJInangamG1ZK+ffC9p3n+1HDNh4kAD/su+dCXlRGwcQSHEGKJuG9YN6K8nyD35UHbK9ZPxiQQiEEL1DQ2syqqriafkZGQEvOxOhC7de0/YwRvpN6WlpRBC2hoaAThWvr+SnhF7/iKFQjBEGBttl/fT0d57NDrr4aMODnfxnJmWplO+H4PgJRSfPBKloa62N+xAcNgBL7dNfGrmFr/ZbGmlr6bJe1lSVdlLTDzEdi3GsguLivaFR5yJPsqUk824c9fF3TPl/FlB7kkns0ynzDKdghBisztWOW1Yab0IlzuQWjZCKL/wtf/uPY+f/dW3Tx/BPwg7efTsWaDv1iGDBuASLCp5Gx4TG70nUE5G+s79P90C99hYzikpK48LC25oarJ18+rfp3d/3W9rkeCeYmCPP27vYOeXv02+e9v5UFhA4umKutq/i4soBOF58ojTobC4jDQOl8N/2SJ0EZ8tbkw5WYTQAH29quoaNpstsD3pkmOxcUw5WctZM4SlbGFM925vb897/jI6Ln7B8tWbvLaVlbP416TT6Vsc7eRkpBFC/XX7VNXUpt25N+vnSQRB9JKQmDJh7JWbGd/PNYguE4r5Eayurx+i3WeZsWnoaqd+auo7409yuJxhOn23L14eaLP6Qf7LSzl3+S9bVUV5wtjRvO2gkDBjo/E0Gk1ge/I5te/qTsYnujo7CvJQfsIW1w2zppmS8dgheZWzKqsMRwx3tl8df/zQ4AH913t48a+poqgw9qdhvO19x6KNDEdU1tQoMd/H4Sgymayq6u/HILDHHyvJyPpY2ajKMRFClqMnlNVWD9Xps3rqTCqFKs4QNR817m7eX7iKb25p2ejhXVxS6uOxWZB78jmJF38znjBW5Z9PhbCULSwHYSdqKsphu/011dUQQssWWxWXlL4tK8Oi3NLa6rE7+G15+ZY1dhzO/5sUUymU78cgsMcfF7LKfn/y8MPYc9Ffb14Xst6PChdxqZjGvrSs3MbWgU6nHz0YKikpIcg9+Zxraelm000FfCiF9yDs5MWr/N9Srn90NHKxzDTLKipt3b3pdNqB7T6S4uLKCvKVNTXv5yxV1YpMuf/AIKytrRd+hJWV1cKFGK4kY48/Jgji0PVLrHc1CKHfcu7qKCm/riiPvXmDw+W0trf/lnPXaMBg/suuq6tfYe802XhiwPatdDod7/FKdkx2fX3Dm5K3Bvgum/VM2eRBXuUEQewOCefNGs4kne+n20dRXp7fY6+hwd5rm/GYUdtdnOl0GkLIyHDExdT0jg5OfWPj9dt3jEaN/FZNDKa1adMmLy+v8PBwKhXnvBF7/LGWgtIvU2dvPxPD4XLlpaRcza16iYlHplxcGxXG4XDG9x80ZegI/ss+k5TMYrFSb95KTb+FEEIEOrw/REqql2D25BOKSkoUmEy844gg3bsrdHvruK13ct7syeFwlBQVArZhuAaRdPU6q6r65r2s9LtZCCGCQCFbPYvLype6uLI7OixNpwwb0P+bu4rl0s7hw4e1tLSmTPmqb1lhRakuhgFWlOriAgCsKPUpPb+iFJ4L7La2tggAgO8OuNUaAAAwCAAAwCAAAACDAAAADAIAADAIAADAIAAAAIMAAAAMAgAAMAgAAMAgAAAAPoP2Pb0Z+SGa5InbmnqTJ37kuj+JY0zqI1XkQdrzVORWTeZzd+LqGjCDAAAATjEAAACDAAAADAIAADAIAADAIAAAAMAgAAAAgwAAAAwCAAC8CPSdlNiT169mZJ66eJkgCFEGw2X5Uv3eOrz97kEhikzZDStsuie72nvZm5clV0/f6NzjHGBfw6qJCT6DEFLVVl7psZQhxuByufEHkp/e+1ugetIz4kJaNqnip88mJSRfoFAIdTVVH3dXWUxZ6njLFtwZBC95fd/uXRcSTqupqgSHHeBTsOhtaXjsmX2em08E7lxuYeaxJ4S3/+T5S4+fP++epoqWkvt+F0OT4R/vnLl0aj+DDyHryzYvvnkh09vG7/DO6LV+v/CT74K9Jz0jLqRlkyr+d97zmLj4k4cOnI05pqmuHn7oiGCWLbgGgT15nU6nb7FbJSctjRDS661d/a6O3dFx/+lf9x4/tehucvzkeca3LmbeS73fuaf/T/0GjRqQlnTrQ4sJQqKXOEJITEKsvbVdoHrSM+JCWjap4v31+l08EysuLt7a2sqqqJCRkhbMskkxiLa2Nv5FsCevqyjIjxlmwNsOjT41YcTw2rr6kOhYXycHSnf/qsfsibuTktU5KZCRl16yfsFBnyMfp0KfCDo9e/n0fRd2bQ5ddzwwlp8oM/LS6EkVF9KyyRanUqm/37ptajH/waPHc2ZOx/gBxFg2BoNIS0szNjaeMmXK5cuXeXuwBG2RlLze0tq6JTj0LavCddUy75D965ctlZPBY94UKmXNDtuTe+Prqus/XOOh09b6rY70PbbezMPfIWilx1JZBRlB6wnZ4kJaNtniCCFjo/Hply/Yr1xuv34TRlmMZWO4SBkREXHu3DkOh7Nu3brW1lYLCwsseZ/KykqPnz3jbeNKXi+rrNy8e6+Oulq4z5a8/MKyisqQ6FguQtW1tRwut6293f2XVd0W1+mvJa/CXLJuPiKQDFOaoBB0Bj0t+ZYIQ+TxH88QQq+eFRYXlPYZqJOT/qfg9KQHxIW0bFLF3xSXVFZXDxsyGCFkPmvGzl+D6+rqcYU8YywbwwyCTqdLS0vLysoeOHDg5MmTd+/exZKzjD15va6h0XGb36RRI32dHek02qB+usnh+04E7owO3Gkx2WTymFH8uANC6NXTAhfzLd7L/Lxt/NKSbt27nnN018nyNywxCbE+g3QQQopq8qpayq/zigSnJz0jLqRlkypeUVXlttX3XV0dQui3lGu6vXVwuQPesjHMINTU1Hbt2rVu3TpJScn9+/evWrWqrq6Of1nsyetJ11NZ1TU3s3LSs3IQQgRCYd4eUpIYFlP5lxlTc2NLiNtB6w1WdBEam805uutkRWmV4PSkZ8SFtGxSxYcbDFm93GalozONRlOQl98X4CeYZRP8nw6w2ewLFy5Mnz5dTEwMIVRZWRkZGenp6fmln2+rq0Lk0JD/CpHGOrsj5ImTuqIU0JNw2tvJE6fQ6eSJi0gxSZlB0Gg0S0vLzpfy8vL/4g4AAAgRcKs1AABgEAAAgEEAAAAGAQAAGAQAAGAQAACAQQAAAAYBAAAYBAAAYBAAAPy4EFgezf4myHsWA5H5XlprqsgTX265izzx0+l74EDvMbicDvLEOW2t5ImLKWrADAIAADjFAAAADAIAADAIAADAIAAAAIMAAAAMAgAAMAgAAMAgAAAAgwAAAPgATZCLIzXWHSHktd2vb58+y5YsxKi5LWifro720rnmDY1NO/aGFr4p4XK5MycbL1swt3uCa7atKnpRfDE2BSF0NDW0qqyat/989JXbKfd424qq8rtPbvN1/LUg97VgNpzUoRRS8UtXUk7ExhEUQowh6rZh3YD+ehjLfvEqPzAkvKGhkUqjem1c31+v7/c2gyA11j2/8LWto/O11N8xahYWFTu4eaVm3OG9jIiOVVJQOBMZFh26J/HSlae5ed8qqKat4hOxeczkkbyXKppKDbWNrku28f7X6Q40Os15xy9UGlVgG07qUAqpeGFR0b7wiIjQ4Pjoo7YrbFzccSZFtLS2Om70WLlkYdzRiF+WLfXc0f0nfTAbREtLC5Zob0RyrHtcQqK52UzTySYYNeMv/mZmOnmy0Tjey00Oq9evXoEQqqiqamezJcW/OcJr2oKf085n3LmexXupb9CXw+Vsi9y8J277PFuzznzD1e7Wv1+4XV9bL7ANJ3UohVRchC7is8WNKSeLEBqgr1dVXcNms3GJ/5GVo6GuOnbUSITQxHFjdm/vfrIWhlOMly9fBgcHS0tLz54928vLi0KheHp6Ghsb8ynbZYQ5rgneFtcNCKG7WTkYD6bNa+wQQlkPHn1wXwrFe3dw2u07xmPHaGmofavgkd0nEUJDRg14r0alPLr77MTeOIaoiGfohqaGpstxN342N6JQKannb821nS2wDSd1KIVUXFVFWVVFmbcdFBJmbDSeRsN2vv/6TbGcrKxvwJ68V6+kevVab2/7X84gfHx8li9fbmho6OzsnJCQcO7cucjISP5lyU5e7xl2bN6QGh/7rq7uUGwcn1Kp524dCzrF6eA0N7ZcjE0ZZfyTjp7m1LmTovyjsZRKXsNJHUrhFUcINbe0bPTwLi4p9fHYjFGWze7IvJc1z3zWqUMHFlrOWevq2d7d6QkGg+BwOIaGhhYWFpMnT2YymZKSkli8UFlZiVVZydvGHuveA9y9/2dlVTVCSFSUYWpslPuC39xQoxljNHXVedsEItjsjokzx4qJi/kf8/z1lK+cgsx6P7ufJhgIYMNJHUrhFS8tK7exdaDT6UcPhkriCJHuREGeqaOpOVBfDyE0afzYDg6n5G3pf2YQOjo6np6eHA4nICAAIRQVFSUvL8+/LKmx7j3A9Vu3ebOGtrb267cyRw4dwqegZh91K3tzgiBEGPTpVj9nXrt3PDjOea6H65Jtrot9qitq922JuJ/xSAAbTupQCql4XV39CnunycYTA7ZvpeOO5B0/2vBtWVnu8xcIofsPH1MohNo/pzP/wTWInTt3pqWlUSjvvUZJScna2pp/WVJj3d//Hf7nOh9W0ff/7/LLSv/QA1Z2ThSCmDRu9CILs27Ocv9ZJis+6tyqzUv3xu+kUCl3rmennc/45OcQf2+HvIaTOpRCKn4mKZnFYqXevJWafot32BzeHyIl1QuLOFNOdq+/r9+ekOaWFhERkWA/3257ECw597XAknPA1122gCXnAAD4MQCDAAAADAIAADAIAADAIAAAAIMAAAAMAgAAwaSLG6Wys7O7/NGRI0dCvwDgRzeI0NDQz3cSBBEdHQ39AoAf3SBiYmKgLwAAoH+5BlFSUrJixYqpU6dWVFTY2NgUFxdDswDgR+OLz2KsWrVqxYoVQUFBycnJCQkJ58+fj42NxfIr295VkvZuCBjRz5kxxo4k5d8yD5B4aArh8h9CjYgU8xtmEDU1NePHj0cIEQSxYMGChoYG6CAAwCnGe0RFRcvKyngPROfk5IiIiECzAOBH44vrQXh4eNjZ2RUVFc2ZM+fdu3chISHQLAAAg3jP4MGDz549W1hYyOFwdHR0YAYBAGAQH6ivrw8PD8/KyqLRaGPHjrWzsxMTE4N+AcAPxRevQXh6elKp1F27dm3fvr2xsdHb2xuaBQAwg3jP69evO2+p9PT0nD17NjQLAGAG8R4dHZ0///yTt52bm6utrQ3NAgCYQSATExOCIFpbW1NSUnr37k2hUPLz87W0tKBZAAAGIXDPYpCRwQ1h0zw27XQseFGUeOISQshsoek0SxMRhsiLv/KDvA90sDsMDAf+stGaSqO2NrcdCDia97T72T+kJlkL6WgKRdldnGKoqampqakpKCj89ddf2dnZ2dnZd+/ePXv2bM9bAxkZ3AjCphFCCGnoqO4+4jPRdAzv5fjJo8wWTXNd6Ws7x0WEITLXZhaVRvX8dX2Q90H7ua6nohLddjl3+3eRmmQtpKMpLGV/8RrE2rVro6Oj9+7dm5GRERIS8urVq543CDIyuBGETSOEEJqzaNrVpLSbV//gvZw82+js8YuNDU0IoZDtUTcu3Oxgdyw0tit4/hohpKKhXFdb1+3fRWqStZCOprCU/UWDKCgoiI6OnjJliq2tbUJCAovF+p9aVVWYo2W2uG6YNc0Ue7RPl5HNP5r4fv+jab9ldOaAqWuryjKl/SM8IxJ/tXGc31DfhBDicDgyclKnUyNXb1hy5uj5bpetqqI8Yexo3jb2JGshHU1hKfuLBsFkMgmC0NHRycvLU1JSamtr+5KPdOLg4MDbEPDrLhA23cW1KBp12Jgh212CHBe4Scn0WrluEW9/bXXdop/t1i3xcvVbo6qpzM+vICnJWkgbLixlf9HI+/btu2PHjkWLFm3atInFYrW3t3f5YytWrBAVFVVUVORyuQUFBVu3bhX8taeUlZUeP3vG2yYjD1oYxatYNZmp91qaWxFCNy7eWmo/T0xcdNjowXfSshFCL3ML8vNe6/TVfFtU1j390rJy503ufXrrHD0YijerVkgbLixlf3EGsW3btunTp+vq6jo7O7NYrD17us53TExM1NXVtbOzi4mJ0dfXj4mJEfyV6SBs+nNuXf/DaOoYuggdITTuZ8PcJy+5XO6mnY79DfohhLT6qKtrq+Y+ftE9cVKTrIW04cJSdhcLxnzrorVsNjswMJDJZGZmZn7NV6TfumCM9w5/3d69v+przq9eMOb2nbv79h/sjGyW6tUL4/AImvi/LBizaYdjwcuixBOXCIJYbDfXePo4giBe/l2w1zeypall0PD+9pttqFRqe1v74b2xj3P++uSff+WCMYeORx+MOqqr2xvxjrWvS7L++gVjhHQ0Ba3sLheM6cIgrK2tu/ro/Y8Th6SkpKSkpJMnT2I3iG+xO1hR6tsMgk9gRanvia81CLIBgwCDAIMQFoOA4BwAAL4IGAQAAGAQAAB8O13cB2FtbU10dTIPyVoAAAaBnJycEELx8fGioqLm5uY0Gu3SpUutra3QLAAAg0CGhoYIocDAwMTERN6eoUOHWlpaQrMA4Efji9cgWltbO5+qyMvLw/j4HQAAQjyD4OHu7m5tba2kpMThcKqrq790qzUAAD+iQYwfPz4tLe358+cEQejp6WF8PhcAAKE/xXj37t327dt3796tqqrq7e397t07aBYAgEG8x9vbe/DgwbW1tRISEoqKiq6urtAsAIBTjPcUFxdbWVmdPn1aRETExcXFzMwM2+8k74kJUp8rIfNBDy6ngzzxy39EkqTsbr6LvLIDznkI4yeK84WVU/AcJx0kiqNvehaDSqXW19fz7pgqLCykUOCeSwCAGcQ/ODk5WVtbl5aWOjo6Pnz40N/fH5oFAGAQ7zEyMho0aNDjx487Ojq2b98uJSUFzQKAH40vnjhYWVnJyclNmjTp559/lpOTmzt3LjQLAGAGgWxsbLKyshBC+vr6vGsQVCrVxMQEmgUAYBDvn9rcuXOnl5cXNAgA4BSjC+bPn+/i4oIQevXq1ZIlS/Lz86FZAAAG8R5vb29zc3OEUJ8+fRwdHT09PaFZAAAG8Z7m5uaJEyfytseNG9fc3AzNAoAfjS9+zSknJ3f69GneDZSXL19mMpk9Xxyp+egIIa/tfn379PmqxA2BKfvSlZQTsXEEhRBjiLptWDegv55gVr5ws1VpfunNs7cY4gwr1wVKGoqIIHKu5fx+Jh0hJCYpZulkrqStRKPTU0+l3r/xQGAbTvZBmHYzw3vnrszrl/HKvniVHxgS3tDQSKVRvTau76/XF/MMYteuXenp6ePHjzc2Nk5PT/fz8+thdyA1Hz2/8LWto/O11N+Fq+zCoqJ94RERocHx0UdtV9i4uHsKYOWKGgr2QXYGE4fwXk5fMa2WVfur7Z59jiFjzcZq6msihBa5L6xh1Qbb7YvcHGm+Zo4UU0owG06qOELo9ZviveEHsUdPtLS2Om70WLlkYdzRiF+WLfXc0f074r9oEKqqqpGRkX/++WdWVlZ4eLiysnIPGwSp+ehxCYnmZjNNJ5sIV9kidBGfLW5MOVmE0AB9varqGowL+eCqfJz5uKwrWQ/TH/Fengs/fzHiEkJImilFo1ObG5vFJMX6Dut7Lfo6QuhdZV3ImtCmuibBbDip4s0tLZ7b/TatW4v9IPwjK0dDXXXsqJEIoYnjxuze7o3zFMPOzi4yMtLExOSTpWtTU1P/XYvD4VRUVCgoKGB5cKPLCHNcE7wtrhsQQnezcrCPDallq6ooq6q8d+qgkDBjo/EY1+nAVXly2DmEUL+f+nXu4XK5iz0WDZkw+MntpxVvKjT0NOpr6iYtmKhvqE+jU9Pjb1a+rRLMhpMqvnP3ngUWc/r27o39IHz9plhOVtY3YE/eq1dSvXqtt7fFaRA7duxACH1Nyub7D9uWLf7+/o8ePdq0aZOMjExjY6O/v//QoUP5fJOk5qOTRw+U3dzS4uXrx6qoPLgvSFgqP7XrdELw2RW+y6baTHl+/7mcslxzQ/P+deFMVebafY4VxRUlL98KYNnkiZ9JTKbRaGYzppWUlmI/CNnsjsx7WYdD9wzU10u/fWetq+eVxFP0bv0t6eLf3Llzp8sfVVNT63J/cXExQmjv3r2HDh3S1tYuLy/fuHHj14R0/juk5qOTB9lll5aVO29y79Nb5+jBULxJ2SRV3m9Ev9L80vrq+vbW9gdpD4dMGJSdkoO4KDslByFU9baq4Gmhpr5mtw2C1IaTJ37hSkpra6vVctu29vaW1lar5bbhewLlMX0VoCDP1NHUHKivhxCaNH6sb2BwydtSbU0NPNcg7t27d+/evYSEhKCgoOzs7AcPHoSGhl6+/D+uslKpVG1tbYQQbxlL/t8kqfno5EFq2XV19SvsnSYbTwzYvhWvO5BX+dCJBlNtpiCEqHTq0EkGLx68rCmvKX5RMtJ0BEJIUlZSa4DWm7w3gtlw8sRjD0ecjTl25vjh8KBABkPkzPHD8vi+KBw/2vBtWVnu8xcIofsPH1MohJpKN68hdjGD2LVrF0LI2tr6woULcnJyCKF3796tWbPmSxINDQ2WlpZNTU0JCQlmZmYBAQGqqqr8v0k5WdkdWz1dNnt2Rphj/zATJKwBQ2rZZ5KSWSxW6s1bqem3EEKIQIf3h0hJ9RLAyjuvzF+IuDjPZa7r4Y1cLvfJ7acZybcRQse2Hp+73nKs2ViCQNeirxW/KBHMhvfAQYgQIhDm45ApJ7vX39dvT0hzS4uIiEiwn2+3/5x8Md3b1NT0ypUrvMuNbW1ts2fPTklJ+ZJKW1tbbm6uqKiotrZ2YmLivHnz/qWgtroqRBKwolSXhZN27QZWlPoc4V1RSkxR46tmEDwmTZq0YsWKqVOncjicq1evTp8+/V+kRUREhgx5/733okWLEAAA3wVfNAgPD4+UlJSsrCyCIFauXPnzzz9DswAADOID8vLyurq6lpaWjx8/hk4BwA/IF+9oOnHixL59+44fP97c3Lx169YjR45AswAADOI9ycnJR44cERMTk5GROXv2bGeQLwAAYBCIQqGIiIjwthkMBpVKhWYBwI/GF69BGBoaBgYGNjc337hx48yZM6NHj4ZmAQDMIN6zefNmLS0tPT29c+fOTZw40c3NDZoFADCDeI+tre3Ro0cXLlwIPQIAmEF8SktLSykJz5kBAPA9zCBqampMTEyYTCaDweByuQRB/M/1IAAA+FEM4vDhw9AdAPjB+eLDWu3t7bGxsXfv3qXRaBMnTpw3bx6uZx/Je1hLSB95EmLIfDpuxBAS0x5zniTBQfgJIlLMb5hBeHl5tbS0LFiwgMPhnD9//vnz5xCNAQBwivGeR48eXb16lbdtYmIya9YsaBYA/Gh88VsMFRWV169f87YrKyuVlJSgWQAAM4j3sNnsOc8hxB4AABnFSURBVHPmjBgxgkaj3b9/X0FBwcbGBv0T7QsAwA9tEE5OTp3bK1euhE4BABjEBwwNDaE7APCDQ4EWAAAABgEAABgEAAD4oAlyceQlr1+6knIiNo6gEGIMUbcN6wb01xOKspGQR917bffr26fPsiX8PiK8I8j9eW5+zOH4oAO+GlqqCCFEEGrqyjl3H4bsPhQQ6s27xZNKperq6bjYef9+7TYchN3TIbikZkl0xVfeal1TW2tuteTkkSgNdbW9YQcam5q83Db9+z/5yrtcC4uKbB3XnYk+ypSTzbhzd2dgUMr5s/+7U193l2s3yv56BE78qw+e/MLX/rv3PH7215pfbL/SILq81Vq7j6bnjvWDh/YPDz4Wczi+c/+AwXp7DvraWK6pYH04ujZ6OsjJy3m6+H2u85W3Wv9QB2GXt1pjPsWorq7G5TjkJa+L0EV8trgx5WQRQgP09aqqa9hsNi5x4U2jJ1U8LiHR3Gym6WQTPnUW2lgkx19O+S39/02DadSdwR6B20I/dofhI4dMnj5xp2cwHIT8lI3hFCMxMbG0tNTY2Hjjxo0MBqOlpcXHx2fs2LF8ypKXvK6qoqz6T1RhUEiYsdF4Gg3bqZbwptGTKr7FdQNC6G5WDp86AT4hCKHR40Z8vNNy4SxWWWX6jf8XOr1hi33o7kPNTc1wEPJTNoaaTp06FRMT4+DgcPDgQR0dnfLyckdHR/4NgtRYd4RQc0uLl68fq6Ly4L4gjLJCmkbfAw0nj6Ur521z+/XjPQY/DZSWlb5yIRUOQj7LxnCKQafTxcXFJSQkNDQ0EEJKSkpYHgxXVlZiVVbytrHHupeWldvYOtDp9KMHQyUlJTCODallC684eegN0KVQKQ+y/1+2k+lM44tJKXAQ8l82BoMwMTFxcHDo27evnZ3d8ePHV61ahWUJbPKS1+vq6lfYO002nhiwfWu3U497vmyhFiePEaMMsu78+cnOn0YPvZd5Hw5C/svGcIrxyy+/ZGVl3b59W1VVtaqqytraetKkSfzLkpe8fiYpmcVipd68lZp+CyGECHR4f4iUVC8s4sKbRt8DUfe41hziog9TaE0d9bfFZZ/8gKaW2ts3ZXAQ8l+24H7N2a1TL1hRqmeBFaW+o4OwJ77mBADgewIMAgAAMAgAAMAgAAAAgwAAAAwCAAAwCAAAwCAAAACDAAAADAIAADAIAAB+XL6rZzFIhdPeTqI4u00Ye0ITkxDS437EYEuSlO/lnCKvbFIf9BBX0oQZBAAAcIoBAAAYBAAAYBAAAIBBAAAABgEAABgEAABgEAAAgEEAAPAD8oOme5MaY336bFJC8gUKhVBXU/Vxd5WVkcGlfCnl+sn4RAIRCKH6hgZWZdXVxFNymPRJFReiUPIeiw7fE3bwRvpNaWkphJC2hkYAvjX1MQ7ld5Xu/ZV0T/krb7X+O+/5Rs+tZ6OPiouLB+8/2NTc5OW68X+Lf+Ot1mx2xyqnDXNmmlrOmoF9gL5e/CtvtRbAxPMub7XGEh3+9bda29iv3bTWccigAV//Zr/1VutvOk6E7FZr8oKVSY2x7q/X7+KZWHFx8dbWVlZFhYyUNBnNORYbx5STJcMdyBAXllDynowOb29vz3v+MjoufsHy1Zu8tpWVswRzKDEYRENDAxnvrcuEYgFX5kGlUn+/ddvUYv6DR4/nzJyOvTO17+pOxie6OjuS0XYyxEltOEbxAJ+Qy+du8GbmnZAUHc6qrDIcMdzZfnX88UODB/Rf7+ElmEOJwSDGjRuXkJCA/e2RF6zcAzHWxkbj0y9fsF+53H79JuydSbz4m/GEsSr/fCoEX1yoQ8mXrpwXFRr98R4s0eFqKsphu/011dUQQssWWxWXlL4tKxPAocRgEPr6+n///beNjU1WVhbGt0desDKpkc1vikv+fPyEt20+a0ZpeXldXT3egb+Wlm423ZSkMzsyxIU3lJy86PAXr/J/S7n+wea4XBqNJoBDicEgGAzG1q1bXV1dY2JiZs+e7efnFx0dzb8secHKpMZYV1RVuW31fVdXhxD6LeWabm8dXImsPOrrG96UvDX4litb/7m48IaSkxcdThDE7pBw3qzhTNL5frp9FOXlBXAoMZgW73uQwYMHh4WF1dfXZ2dnFxQU8C9LXrAyqTHWww2GrF5us9LRmUajKcjL7wvww/thKyopUWAyqVRSQlxJEheuUPKeiQ7X7a3jtt7JebMnh8NRUlQI2OYlmEOJ4WvO5ORkCwuLr/95WFGqC3FYUapngRWlPoesrzm/yR0AABAi4FZrAADAIAAAAIMAAAAMAgAAMAgAAMAgAAAAgwAAAAwCAAAwCAAAwCAAAACDAAAA+AzBXZOyG5x3iyGv7JlbyXzkhEKQqE0TIUuZTievbFIfTCKP8+4kPqw1J2AxeeIMGUWYQQAAAKcYAACAQQAAAAYBAAAYBAAAYBAAAIBBAAAABgEAABgEAAA/IDRBLg57Zvwfb57cK3lKp9AUJGRn95uAELqQd6u0oVKESh+uojdafTCWstMybkccP0mlUKR6SW51dVFTUcHVEIyx7l8s/maG985dmdcvC/hQ/r+2XEk5ERtHUAgxhqjbhnUD+usJpnjS32lKEsxxmgYcLvfqy8wX1W+4XO44DYORagMRQlVN75Jzf29qb2HQ6Jb9TRTEZQWhbME1iJraWu8d/p2x7sFhB/jMjM+vKbld9NB+hGUvhsTDsufnctNFqHQRKn396EUdnI7YJ1dlRaX05LX4LLu1tc3Lb3fCsUg1FZXYhKTAkAOhATtw9WSW6ZRZplPQP7HuK60X4XWH12+K94YfxH73Pfah/JjCoqJ94RFnoo8y5WQz7tx1cfdMOX9W0MQrGmsuPc8ori9XkmAihLLfPqtqrnM2XNjCbot6kKTaS0FNSvHsXzfGahgMVtJ9UVUU9zTFyXChIPREcE8xsGfGv62v6COn3oshgRAaoKCTV/W6pJ41VLkfQohKoeoxtZ5V5PNfNofTgRCqb2hECDU1NzMYpDwHwX+s++c0t7R4bvfbtG6t4A/lx4jQRXy2uDHlZBFCA/T1qqpr2Gy2oInfK3k6XEV/kIIu7+XfFQXDlfUIghCjMwYr6j4qf17X2ljZXDtYSRch1Jep2dbBLq2vFISeYJ5BtLW1cTgcUVFR/qW6jHXnZ2qqLqV4t/hpbUuDjKjkg9LcDg5HQ0r5YVmeprQym9PxrCKfSmCwSzExsS0bnJc5rpORluZwOMf278X+eePFuscdjcAru3P3ngUWc/r27o29YOxD+TGqKsqqKsq87aCQMGOj8RhTcHGJz+o3ASH0qqaY9/Jda4O0qCRvW5ohWd5Y/a6loZfIh5gyaYZEXWuDSi/5/7wnGD4SBQUFzs7OGzdufPjw4ezZs2fOnHn5MoYzWOyx7toyqsbaI049uXIwO5FCUMTojGm6YxAiDmQnnH6aoiunQcURG/8yvyDqxMnk6CMpZ0+tXLpwo7cv9s8bllj3TziTmEyj0cxmTPs4mRIX2Ieyy+nPRg/v4pJSH4/N2OvHLv7JSRyFID5vO0EQglA2BoPw9vZeuHDh1KlT7ezsoqOjL168eOLECf5lsce6t7LbtWVUHEfOdxg5d4CCDkKotaN9mu4Yp1FWy4fOIhBiikljmE5n3x82eBDPv63MzV4VFPKSvjGCJdb9Ey5cSXn2d67Vctu1m9xbWlutlttWVmF7Kh/7UH5CaVm5ja0DnU4/ejBUUhJzXCgZ4jKivepbm3jbda2NUgxJGVHJ+rbGzh/g7RSEsjEYBJvNHjt27NSpU2VkZJSUlMTFxbHM8bDHute3NR7583wruw0hlF54f4hS3+ySZ6kFWQihhramnLd/D1Hqy3/Z/fvp3n/0uLqmBiGUlpGppqoiLSWF8XjFFev+CbGHI87GHDtz/HB4UCCDIXLm+GF5JhOXOPah/Ji6uvoV9k6TjScGbN9Kx70+BUni+vLaD8pyOVxOc3vrE9bLAfI6UgxJppj0E9ZLhNCLqiKCoChLMgWhbAyfZDU1NRcXl46ODgkJib1790pKSiooKPAviz3WXV5cZqLW8IicJIS4mtIqs/UmdHA4Z/9KDbt3BiFkojNSTQpD2SOHDbVZON92vasInS4t1WuvH+ZTDFyx7v8C72tUjGAfyv93cpSUzGKxUm/eSk2/xav+8P4QKaleAije2VhDtYE1zXXh2fEdHI6h2kAtGRWE0IKBU87lpqcX3qdTaAsHTRWQsjGsKMVms2/evKmtrS0hIXH8+HFpaelly5b9yyUoWFGqq5kcrCj1+ZULWFHqU3p+RSkMMwgajfbzzz/ztt3d3REAAN8LcKs1AABgEAAAgEEAAAAGAQAAGAQAAGAQAACAQQAAAAYBAAAYBAAAYBAAAIBBAAAAfAaBfQHC/wl5D2uRCqnPDrXV1pAnzpCTJ0m5/NY98spWMholjMdJ0aWb5Invi7xFnvj+m/tgBgEAAJxiAAAABgEAABgEAABgEAAAgEEAAAAGAQAAGAQAAGAQAAD8gNAEuTjyMuOFMY1+W9A+XR3tpXPNGxqbduwNLXxTwuVyZ042XrZgroC3JfPJk8C4Uxf8djW2tOw5E1fEYiHEnfLTCCuTnwV/NLGLH0757fbfT6XExBFCavLyTrMs9l1ILK6sQFxkYjBs3riJ3ZNd6r74bf7btPh0UXHGYrdFypqKCBFZ17JvnE5DCPUdpmvhMIdCoTTWNSbuT36bXyr0MwheZvy+3bsuJJxWU1UJDjsg+Mron+T1iNDg+OijtitsXNw9cWgWO7h5pWbc4b2MiI5VUlA4ExkWHbon8dKVp7l5gtyW4oqKqEsXEJeLEDp+9bKCjMxh183717lc/OPO369fC/hokiH+d3GR+7xFoXZOoXZObnMXxaRdV5CSPuCwPni14+Wce7nFRd8qqKSp6BTsOGyiAe/lzFUzalm1/it2/2ofPH7OOK3+WqLiDNvtK5IPnA+w/fXM3rMrty2nUClCbxDkZcYLXRp9/MXfzEwnTzYax3u5yWH1+tUrEEIVVVXtbLakuITAtqWlrS3gVKzDHHPeyzXmlnazzRBCVXXv2tkdEjhS4EkdTezi7R3s/LK3SXcy1kaE+sfHVryrtZs+e9XUGQih6vo6dkd3emJkMf6Py/cepD/kvUwMS04+cB4hJC0vTaNRWxqbFdQVmhuaXzx8iRBivWG1NLXoDNT+D04xuFwu/5HEnZCXGS90afSb19ghhLIePPrg6xSK9+7gtNt3jMeO0dJQE9iG7zubYDZ2rI6yyseVB5w6mfH48bhBgzUUFQV8NLGLV9fXG+j0WT7ZVFVOPvHOrR1xMaF2ThSCEpR85s5fz8b0H6DO/Ob8x4SQJISQ/oh+H38SbTyXDDUyeJTxuLyIxRBjMMQYej/1y7v/XFNfQ0VbWZr5tTnVGGYQRUVFq1atMjY2HjRo0IIFCzZu3FhRUYHBbkjLjBf2NHoeOzZvSI2PfVdXdyg2TjDbcj7zNo1KnTrS8BNd98VLE7fvrGtqirmWIuCjiV1cSUZ22+LlqnLyCKG5Y41Ka6pYtTUIoU0WVqc2e9U1NZ2+mYql8mi/WDczTwkpienLTFubW6M8j5haT3E7vGnklBHPH7z4+lktBoPw9fX18vL6/fffY2NjR40atWLFCk9PDCfe5GXGC3UaPULo7v0/K6uqEUKiogxTY6PcF68Esy3Xc7Lz3hTZBwd5Ho5qaW+3Dw66lpNdVfcOISQqImIybNiLkhIBH03s4oXlZWmP//zoTz16+rqgur4OISRKF5k4yOBl2Vs+a9YfoSclJ4UQam9tv5/6QKOfOkKotbk1dH14oG1QYliyvJp8ZUllzxlEQ0ODjo4OQmjo0KEPHjwYNGhQXV0d/7LkZcYLbxr9+w/erdu8WUNbW/v1W5kjhw4RzLbsX+dyaNPmiA2b/G1/EaXTIzZsevLqVcy1awihNjY7/dHDYbq6Aj6a2MUJgoi6epE3a7iU/UdvZZWnrwtP3UxFCLWz2Rl/PTHQ7sPnrxhuPHT6clOEEI1OHWY8NO/BC4SQQ6AdzymGTTLoaO/4+m8xMFyDUFdX37p1q5GRUXp6+qBBg9LT08XExPiXJS8zXljT6P+5vOPyy0r/0ANWdk4Ugpg0bvQiCzPBbwsPO7M5IWcTbH/dTSGIcYMHWxpNFPCysYtrKSrZTzfbdvoEl8uVl5LePHehOIMRdinZ8eA+CkGM0R84Z/S47l4BfL+RdOD8wo0LthzbzOFwH99+cjPxFkLo+I7oxa5WVBr1XVVdlNeRbzjo+F9Rqq2tLSEh4eXLl/379587d+6TJ0+0tLRkZWW/+POwotTnPYEVpT4/XYcVpT6j51eUwjCDEBERWbJkSefLoUOHIgAAvgvgVmsAAMAgAAAAgwAAAAwCAAAwCAAAwCAAAACDAAAADAIAADAIAADAIAAA+HH5D9K9AQCAGQQAAGAQAACAQQAAAAYBAAAABgEAABgEAAD8ILjRe1wud9u2bXl5eSIiIn5+fhoaGth/xaNHj4KCgmJiYjBqstnsLVu2lJSUtLe329vbm5iYYBTncDheXl4FBQUUCsXX11cXx6Kvn1BVVTV37txjx47xFiLGiKWlpaSkJEJIXV3d398fo3JUVFRaWlp7e/vixYvnzp2LUTk5OTkpKYkgiNbW1tzc3MzMTN5bwHWouLm5lZSU0Gi0HTt2YGx4W1ubh4dHcXGxpKSkj4+PpqYmX59DweTatWvu7u5cLvfhw4cODg7Y9Q8dOjRr1iwrKyu8somJif7+/lwut7a2dtKkSXjFr1+/vmXLFi6Xe+/ePTJ60t7evmbNGlNT0/z8fLzKra2tFhYWZBwn9+7ds7e353K5jY2NYWFhJB2Nvr6+8fHxeDVv3Lixfv16LpebmZnp5OSEUfnkyZPe3t5cLjc/P3/lypX8SAnuKcb9+/cnTJiAEDIwMHj69Cl2fS0trfDwcOyy06dPX7duHe+vPf+ZWp8wefLkHTt2IIRKSkqkpaWxFx8YGLho0SJFHIFXn5Cbm9vU1LRq1arly5c/evQIo/Lt27f79evn6Ojo4OBgbGxMxqH45MmTly9fzp8/H6+strZ2R0cHl8utr6/Hm5Dw8uVLIyMjhJCOjk5+fv73eYrR0NDQq9f71eJpNBqHw6FQcNrZlClTSnAEt3wCb8n/hoaGdevWubi44L9oRKG4u7vfuHEjNDQUr3JSUhKTyRw3blxERAT2skVFRVetWjV//vzCwsLVq1enpKTgGs2ampq3b99GRka+efPGwcHh6tWr2IuPiopau3YtdlkJCYni4uJp06bV1tZGRkZiVO7fv396evrkyZMfPnzIYrH4ycQU3BmEpKRkY2Nj57k3XncgldLS0mXLlllYWMyYMYMM/YCAgJSUFC8vr5aWFrwGkZmZaW1tnZub6+bmVlWFM51AW1vbzMyMtyEjI4MlnJGHjIzMhAkTaDSajo4Og8Gorq7G2+36+vrCwkJDQ0Ps43j8+PEJEyakpKRcuHDBzc2tra0Nl/LcuXMlJCSWLFmSmpo6cOBAfhJzBfdTN3z48Js3byKEHj582K9fP/IuheIVrKysXLVqlaurq4WFBfZqz58/HxUVhRBiMBgUCgWvaZ48eTImJiYmJkZfXz8wMJDJZGIUT0xMDAgIQAiVl5c3NjYqKCjgUv7pp58yMjJ4yi0tLf8SyNI9srOzR48eTcaxJy0tzbvk2atXLzabzeFwMJ4TjRkzJjY21tTUlM+r+4J7ijFlypTMzMyFCxcihHbt2kXSb8EYR84jMjKyrq7uwIED4eHhBEEcPnxYREQEl/jUqVM9PDyWLl3KZrM9PT0xKpPaE4TQvHnzPDw8Fi9eTKFQ/P39MVrbpEmTcnJy5s2bx+VyfXx8sBdfUFBAxjdoCKFly5Zt2bJlyZIlbDZ748aNoqKiuJS1tLRCQkIiIiKkpKT8/Pz4OhjgaU4AAITvFAMAADAIAADAIAAAAIMAAAAMAgAAMAgAAMAggB+ThoaGNWvW8KNQUlLypcdV9+/fv3///m7/cwAMAviPqa2tzc3N5VOEzxuTyLgpCwCDADDg5+fHYrGcnJxKSkqmTZu2ZMmSlStXJicne3h48H7A2to6OzsbIRQVFWVpaWlubh4UFNSl1PPnz21sbObPn29iYnLy5EnezsePHy9YsGD27NnR0dG8Pf9TBwCDAAQFLy8vRUXFsLAwhNDr16+DgoKOHj36+Y9lZGQ8e/YsMTExOTm5rKzs4sWLn//M2bNnHR0dExISTpw4ERwczNtZWVkZExNz+vTp2NjY3Nzcr9EBBA0atABACDGZTBUVlS7/0507d548eWJpaclb90VNTe3zn3F3d8/IyIiKisrLy2tububtnDFjBoPBYDAYJiYmWVlZpaWln+gMHz4cOg8GAQgBDAbj84sCbDYbIcThcGxsbJYvX44QamhooFKpn//zdevWycjIGBsbz5gx4/Lly++PrX/Wy+FwOHQ6ncvlfqKD/dFsAE4xAHx/HGi0jo4O3nbnM3uysrKvXr1CCL158yYvLw8hNHr06AsXLjQ1NbHZbAcHh5SUlC5nGc7OzryZQqfa1atX29ra3r17l56ePmrUqFGjRn2uA88KwgwCENzTCmVl5WXLlvn7+3dOHMaMGZOYmDht2rTevXuPGDECIWRsbJyXl7dgwQIOh2NkZGRubv65lJOT06JFi6SkpHR0dNTV1YuLixFCampqixYtamtrs7e37927d+/evT/RKSkpgW8xBBx43BsAADjFAAAADAIAADAIAADAIAAAAIMAAAAMAgAAMAgAAMAgAAD47vk/XObvpCo8rMoAAAAASUVORK5CYII=" />

As we might expect from the cluster centers we visualized before, the main point of confusion is between the eights and ones. But this still shows that using k-means, we can essentially build a digit classifier without reference to any known labels!

Just for fun, let’s try to push this even further. We can use the t-distributed stochastic neighbor embedding (t-SNE) algorithm (mentioned in “In-Depth: Manifold Learning” on page 445) to preprocess the data before performing k-means. t-SNE is a non‐linear embedding algorithm that is particularly adept at preserving points within clusters. Let’s see how it does:

In [ ]:
from sklearn.manifold import TSNE
# Project the data: this step will take several seconds
tsne = TSNE(n_components=2, init='pca', random_state=0)
digits_proj = tsne.fit_transform(digits.data)
# Compute the clusters
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits_proj)
# Permute the labels
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]
# Compute the accuracy
accuracy_score(digits.target, labels)

Output:<br>`
0.93356149137451305
        `

That’s nearly 94% classification accuracy without using the labels. This is the power of unsupervised learning when used carefully: it can extract information from the dataset that it might be difficult to do by hand or by eye.

# Example 2: k-means for color compression

One interesting application of clustering is in color compression within images. For example, imagine you have an image with millions of colors. In most images, a large number of the colors will be unused, and many of the pixels in the image will have similar or even identical colors.

For example, consider the image shown in Figure 5-120, which is from Scikit-Learn’s datasets module (for this to work, you’ll have to have the pillow Python package installed):

In [ ]:
# Note: this requires the pillow package to be installed
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(china);

The image itself is stored in a three-dimensional array of size (height, width, RGB), containing red/blue/green contributions as integers from 0 to 255:

In [ ]:
china.shape

Output:<br>`
(427, 640, 3)
        `

> **Figure 5-120: _The input image_**<br><img width="300" src="data:image/png;base64, ..." />

One way we can view this set of pixels is as a cloud of points in a three-dimensional color space. We will reshape the data to [n_samples x n_features], and rescale the colors so that they lie between 0 and 1:

In [ ]:
data = china / 255.0 # use 0...1 scale
data = data.reshape(427 * 640, 3)
data.shape

Output:<br>`
(273280, 3)
        `

We can visualize these pixels in this color space, using a subset of 10,000 pixels for efficiency (Figure 5-121):

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    # choose a random subset
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))
    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))
    fig.suptitle(title, size=20);

In [ ]:
plot_pixels(data, title='Input color space: 16 million possible colors')

> **Figure 5-121: _The distribution of the pixels in RGB color space_**<br><img width="300" src="data:image/png;base64, ..." />

Now let’s reduce these 16 million colors to just 16 colors, using a k-means clustering across the pixel space. Because we are dealing with a very large dataset, we will use the mini batch k-means, which operates on subsets of the data to compute the result much more quickly than the standard k-means algorithm (Figure 5-122):

In [ ]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]
plot_pixels(data, colors=new_colors,
            title="Reduced color space: 16 colors")

> **Figure 5-122: _16 clusters in RGB color space_**<br><img width="300" src="data:image/png;base64, ..." />

The result is a recoloring of the original pixels, where each pixel is assigned the color of its closest cluster center. Plotting these new colors in the image space rather than the pixel space shows us the effect of this (Figure 5-123):

In [ ]:
china_recolored = new_colors.reshape(china.shape)
fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(china)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(china_recolored)
ax[1].set_title('16-color Image', size=16);

> **Figure 5-123: _A comparison of the full-color image (left) and the 16-color image (right)_**<br><img width="300" src="data:image/png;base64, ..." />

Some detail is certainly lost in the rightmost panel, but the overall image is still easily recognizable. This image on the right achieves a compression factor of around 1 million! While this is an interesting application of k-means, there are certainly better way to compress information in images. But the example shows the power of thinking outside of the box with unsupervised methods like k-means.